In [ ]:
!pip install markdown

In [3]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-08-13 03:58:47--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3495 (3.4K) [text/plain]
Saving to: ‘chat_assistant.py’

chat_assistant.py   100%[===================>]   3.41K  --.-KB/s    in 0s      

2025-08-13 03:58:47 (42.9 MB/s) - ‘chat_assistant.py’ saved [3495/3495]



In [ ]:
!pip install fastmcp

In [1]:
import chat_assistant
from openai import OpenAI

## Tools list

In [2]:
import random

known_weather_data = {
    'berlin': 20.0
}


In [3]:
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [4]:
#description of a tool
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Returns the temperature for the selected city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [5]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Updates the temperature for the selected city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city"
            },
            "temp": {
                "type": "integer",
                "description": "The temperature to update"
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

## Chat initialization

In [6]:
tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)
tools.add_tool(set_weather, set_weather_tool)

tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Returns the temperature for the selected city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The name of the city'}},
   'required': ['city'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'Updates the temperature for the selected city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The name of the city'},
    'temp': {'type': 'integer', 'description': 'The temperature to update'}},
   'required': ['city', 'temp'],
   'additionalProperties': False}}]

In [7]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

In [8]:
client = OpenAI()

In [9]:
chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

## MCP

In [16]:
from fastmcp import Client
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        result = await mcp_client.list_tools()
        # print(result[0])